# Modelo Matemático - Fábrica de camisetas

### Alunos: 
    Arthur Henrique Sousa Cruz        Matrícula: 201611484
    Pedro Silveira Lopes              Matrícula: 201611481

## Objetivos

Este documento visa propor um modelo matemático para o problema da Fábrica de camisetas, além de solucioná-lo utilizando a linguagem de programação Julia.

## Enunciado

Uma certa fábrica de camisetas deseja aproveitar as finais de um campeonato de futebol para vender camisetas dos times envolvidos. Os jogos vão durar quatro semanas. O custo de produção de cada camiseta é de $R\$$ $2,00$ nas duas primeiras semanas e $R\$$ $2,50$ nas duas últimas, quando a concorrência demandar por material no mercado. A demanda semanal de camisetas será de $5.000$, $10.000$, $30.000$ e $60.000$. A capacidade máxima de produção da empresa é de $25.000$ camisetas semanalmente. Na primeira e na segunda semanas, a empresa poderá contratar horas extras de serviço e fabricar mais $10.000$ camisetas em cada semana. Nesse caso, o custo de produção sobe para $R\$$ $2,80$. O excesso de produção pode ser estocado a um custo de $R\$$ $0,20$ por unidade por semana. Formule um modelo que minimize os custos.

### Implementação - importações

O bloco abaixo importa os pacotes necessários para resolução do problema e declara o modelo utilizando o solver Gurobi.

In [1]:
#!/usr/bin/env  julia
using JuMP
using Gurobi

m = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

## Dados

Como dados de entrada temos a quantidade de semanas, os custos de produção de cada semana, as demandas semanais, o custo de produção utilizando horas extras de trabalho, o custo de estoque de camiseta por semana, a capacidade máxima de produção e a capacidade máxima de produção com o uso de horas extras.

- $semanas$: lista de índices de semanas (variando de 1 até 4);
- $custos$: lista com os custos, sendo $custos_{i}$ o custo da semana $i$;
- $demandas$ = lista com as demandas, sendo $demandas_{i}$ a demanda da semana $i$;
- $custo\_por\_excesso$ = custo de armazenamento de cada camiseta;
- $custo\_hora\_extra$ = custo de produção adicional utilizando horas extras de trabalho;
- $capacidade\_maxima$ = capacidade máxima de produção;
- $capacidade\_extra\_max$ = capacidade máxima de produção durante as horas extras.

### Implementação - Declaração dos dados

Abaixo está o bloco com a declaração dos dados constantes do problema.

In [2]:
# Dados
semanas = collect(1:4)
custos = [2, 2, 2.5, 2.5]
demandas = [5000, 10000, 30000, 60000]
custo_por_excesso = 0.2
custo_hora_extra = 0.8
capacidade_maxima = 25000
capacidade_extra_max = 10000

10000

## Variáveis

Como variáveis temos a quantidade de camisetas a serem produzidas por semana durante o horário normal de trabalho e a quantidade de camisetas a serem produzidas por semana durante o horário extra de trabalho.

- $quantidades$: lista com as quantidades de camisetas a serem produzidas em cada semana em horário normal de trabalho, onde $quantidades_{i}$ é a quantidade de camisetas que deve ser produzida na semana $i$;
- $quantidades\_extras$: lista com as quantidades de camisetas a serem produzidas em cada semana nos horários extras de trabalho, onde $quantidades\_extras_{i}$ é a quantidade de camisetas que deve ser produzida na semana $i$ durante as horas extras;


### Implemetação - Variáveis

O bloco abaixo implementa a declaração das variáveis.

In [3]:
# Variaveis
@variable(m, quantidades[semanas] >= 0, Int)
@variable(m, quantidades_extras[semanas] >= 0, Int)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Array{VariableRef,1}:
 quantidades_extras[1]
 quantidades_extras[2]
 quantidades_extras[3]
 quantidades_extras[4]

### Abreviações

Abaixo está o modelo matemático proposto para o problema. Para facilitar a compreensão do modelo foram usadas as seguintes abreviações:

- $s$ $=$ $semanas$
- $cm$ $=$ $capacidade\_maxima$
- $cem$ $=$ $capaciadde\_extra\_max$
- $q$ $=$ $quantidades$
- $c$ $=$ $custos$
- $qe$ $=$ $quantidades\_extras$
- $che$ $=$ $custos\_hora_extra$
- $d$ $=$ $demandas$
- $cpe$ $=$ $custo\_por\_excesso$

## Função objetivo

Abaixo está a função objetivo. A primeira multiplicação calcula o custo de produção nos horários de trabalho normais, a segunda calcula o custo das produções nas horas extras enquanto o somatório em $j$ calcula os custos de estoque.

$$
\begin{equation}
    \min f(q, qe) = \sum_{i \in s}((q_{i} c_{i}) + (qe_{i} (c_{i} + che)) + \sum_{j \in s}((q_{j} + qe_{j}) - d_{j}) ce)
\end{equation}
$$

### Implementação - Função objetivo

Abaixo está a implementação da função objetiva proposta no bloco acima.

In [4]:
# Funcao Objetivo
@objective(m, Min, sum(
                        quantidades_extras[i] * (custos[i] + custo_hora_extra)
                        +
                        quantidades[i] * custos[i]
                        +
                        (sum(
                            quantidades[j] + quantidades_extras[j] - demandas[j]
                         for j in collect(1:i)
                        )
                        *
                        custo_por_excesso)
                    for i in semanas
                   )
           )

3.6000000000000005 quantidades_extras[1] + 2.8000000000000007 quantidades[1] + 3.4000000000000004 quantidades_extras[2] + 2.6000000000000005 quantidades[2] + 3.7 quantidades_extras[3] + 2.9000000000000004 quantidades[3] + 3.5 quantidades_extras[4] + 2.7 quantidades[4] - 34000

## Restrições

Abaixo estão as restrições dadas pelo problema. A duas primeiras garantem que a quantidade de camisetas produzidas na semana não ultrapassará a capacidade máxima de produção. A terceira restrição garante que nas semanas $3$ e $4$ não haverá produção por horas extras (já que não devem ser contratadas horas extras nesta semanas). Por fim, a última restrição garante que a demanda será atendida.

$
\begin{equation}
    q_{i} \le cm, \forall i \in s; \\
    qe_{i} \le cem, \forall i \in s; \\
    qe_{i} = 0, i \in \{3, 4\}; \\
    \sum_{j = 1}^{i}(q_{j} + qe_{j}) >= \sum_{j = 1}^{i}(d_{j}), \forall i \in s; \\
\end{equation}
$


### Implementação - Restrições

Abaixo está a implementação das restrições definidas acima.

In [5]:
# Restricoes
@constraints(m,
    begin
        [i in semanas], quantidades[i] <= capacidade_maxima
        [i in semanas], quantidades_extras[i] <= capacidade_extra_max
        [i=3:4], quantidades_extras[i] == 0
        [i in semanas], sum(quantidades[j] + quantidades_extras[j] for j in collect(1:i)) >= sum(demandas[j] for j in collect(1:i)) 
    end
)

## Implementação - Modelo final

O código abaixo imprime o modelo gerado.

In [6]:
# Imprime modelo
println(m)

Min 3.6000000000000005 quantidades_extras[1] + 2.8000000000000007 quantidades[1] + 3.4000000000000004 quantidades_extras[2] + 2.6000000000000005 quantidades[2] + 3.7 quantidades_extras[3] + 2.9000000000000004 quantidades[3] + 3.5 quantidades_extras[4] + 2.7 quantidades[4] - 34000
Subject to
 quantidades[1] integer
 quantidades[2] integer
 quantidades[3] integer
 quantidades[4] integer
 quantidades_extras[1] integer
 quantidades_extras[2] integer
 quantidades_extras[3] integer
 quantidades_extras[4] integer
 quantidades[1] ≥ 0.0
 quantidades[2] ≥ 0.0
 quantidades[3] ≥ 0.0
 quantidades[4] ≥ 0.0
 quantidades_extras[1] ≥ 0.0
 quantidades_extras[2] ≥ 0.0
 quantidades_extras[3] ≥ 0.0
 quantidades_extras[4] ≥ 0.0
 quantidades_extras[3] = 0.0
 quantidades_extras[4] = 0.0
 quantidades[1] + quantidades_extras[1] ≥ 5000.0
 quantidades[1] + quantidades_extras[1] + quantidades[2] + quantidades_extras[2] ≥ 15000.0
 quantidades[1] + quantidades_extras[1] + quantidades[2] + quantidades_extras[2] + qua

## Solução

Abaixo está explicado o restante da implementação do modelo junto a seu código fonte.

### Implementação - Execução do solver

A execução do código abaixo irá obter os resultados que satisfazem o modelo.

In [7]:
start_time = time()
optimize!(m)
end_time = time()

Academic license - for non-commercial use only
Optimize a model with 14 rows, 8 columns and 30 nonzeros
Variable types: 0 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 1e+05]
Presolve removed 14 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 258000 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.580000000000e+05, best bound 2.580000000000e+05, gap 0.0000%


1.55648128436743e9

### Implementação - Resultados

A execução do código abaixo irá imprimir os resultados obtidos com o Gurobi.

In [8]:
println("Tempo: $(end_time - start_time) s")
println("Resultado função: ", JuMP.objective_value(m))

for i in semanas
    if (JuMP.value(quantidades[i]) > 0)
        println("quantidades[", i, "]: ", JuMP.value(quantidades[i]))
    elseif (JuMP.value(quantidades[i]) == 0)
        println("quantidades[", i, "]: ", 0.0)
    end
    if (JuMP.value(quantidades_extras[i]) > 0)
        println("quantidades_extras[", i, "]: ", JuMP.value(quantidades_extras[i]))
    elseif (JuMP.value(quantidades_extras[i]) == 0)
        println("quantidades_extras[", i, "]: ", 0.0)
    end
end

Tempo: 9.2256600856781 s
Resultado função: 258000.0
quantidades[1]: 25000.0
quantidades_extras[1]: 0.0
quantidades[2]: 25000.0
quantidades_extras[2]: 5000.0
quantidades[3]: 25000.0
quantidades_extras[3]: 0.0
quantidades[4]: 25000.0
quantidades_extras[4]: 0.0
